# Time Decay
**Truth Premise: When a day ends and a new day begins in life of options premium, its value depreciates because of time decay because it gets closer to expiry and the probability of its contract going as expected by the buyer gets lesser.**

## Strategy
1. Short both put and call at day closing right before the expiry
2. Square of in the morning

## Pseudocode
```python
nifty_at_0320
at_the_money_call_strike
at_the_money_put_strike
call_strike_premium_at_0320
put_strike_premium_at_0320
call_strike_premium_at_0915
put_strike_premium_at_0915
call_strike_premium_at_0916
put_strike_premium_at_0916
call_strike_premium_at_0917
put_strike_premium_at_0917
call_strike_premium_at_0918
put_strike_premium_at_0918
pnl_0915
pnl_0916
pnl_0917
pnl_0918
```

In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
pickle_file_name = "ocdf_2024_02_17.pkl"
# pickle_file_name = "test_analyzer_ocdf_2024_02_17.pkl"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)
# train_dates = test_dates

@cache
def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

@cache
def get_symbol_first_candle(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open, data.iloc[0].high, data.iloc[0].low, data.iloc[0].close

@cache
def get_first_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].close

@cache
def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_closest_expiry(SYMBOL, row.name), axis=1)
train_dates = train_dates.loc[train_dates.expiry == train_dates.index + pd.Timedelta(days=1)]

day_before_expiry = 1

def get_nifty_price(trade):
    data = ut.get_data(symbol=SYMBOL, date=trade.trade_day_before_expiry.date(), interval=INTERVAL, exchange=EXCHANGE)
    t = dt.time(hour=15, minute=20)
    try:
        return data.loc[data.index.time == t].iloc[0].open
    except IndexError:
        return pd.NA
    except AttributeError as e:
        return pd.NA

train_dates = train_dates.copy()
train_dates["trade_day_before_expiry"] = train_dates.expiry - pd.Timedelta(days=day_before_expiry)
train_dates.loc[:, "nifty_at_0320"] = train_dates.apply(get_nifty_price, axis=1)
train_dates = train_dates.loc[train_dates.nifty_at_0320.notna()]
train_dates["atm_strike"] = train_dates.apply(lambda trade: round(trade.nifty_at_0320 / 50) * 50, axis=1)
# train_dates["call_atm_strike"] = train_dates["atm_strike"] - 100
# train_dates["put_atm_strike"] = train_dates["atm_strike"] + 300



In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if pr.shape[0] == 0:
        return pd.NA
    # x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    # if x.shape[0] == 0:
    #     print(tm)
    #     print(td)
    #     print(pr.loc[(pr.index.date == td)])
    #     print(trade)
    return pr.loc[(pr.index.date == td) & (pr.index.time >= tm)].iloc[0].close

# train_dates["call_at_0320"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.name.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=20)), axis=1)
train_dates["put_at_0320"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.name.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=20)), axis=1)
# train_dates["call_at_0915"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.expiry.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=15)), axis=1)
train_dates["put_at_0915"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=15)), axis=1)
# train_dates["call_at_0916"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.expiry.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=16)), axis=1)
train_dates["put_at_0916"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=16)), axis=1)
# train_dates["call_at_0917"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.expiry.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=17)), axis=1)
train_dates["put_at_0917"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=17)), axis=1)
# train_dates["call_at_0918"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.expiry.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=18)), axis=1)
train_dates["put_at_0918"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=18)), axis=1)
train_dates["put_at_10"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=10, minute=0)), axis=1)
train_dates["put_at_11"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=11, minute=0)), axis=1)
# train_dates["put_at_12"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=12, minute=0)), axis=1)
# train_dates["put_at_13"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=13, minute=0)), axis=1)
# train_dates["put_at_14"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=14, minute=0)), axis=1)
# train_dates["put_at_15"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=0)), axis=1)
# train_dates["put_at_1520"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=20)), axis=1)

AttributeError: 'Series' object has no attribute 'put_atm_strike'

In [50]:
# train_dates["call_pnl_0915"] = train_dates["call_at_0320"] - train_dates["call_at_0915"]
# train_dates["call_pnl_0916"] = train_dates["call_at_0320"] - train_dates["call_at_0916"]
# train_dates["call_pnl_0917"] = train_dates["call_at_0320"] - train_dates["call_at_0917"]
# train_dates["call_pnl_0918"] = train_dates["call_at_0320"] - train_dates["call_at_0918"]

train_dates["put_pnl_0915"] = train_dates["put_at_0320"] - train_dates["put_at_0915"]
train_dates["put_pnl_0916"] = train_dates["put_at_0320"] - train_dates["put_at_0916"]
train_dates["put_pnl_0917"] = train_dates["put_at_0320"] - train_dates["put_at_0917"]
train_dates["put_pnl_0918"] = train_dates["put_at_0320"] - train_dates["put_at_0918"]

# train_dates["pnl_0915"] = train_dates["call_pnl_0915"] + train_dates["put_pnl_0915"]
# train_dates["pnl_0916"] = train_dates["call_pnl_0916"] + train_dates["put_pnl_0916"]
# train_dates["pnl_0917"] = train_dates["call_pnl_0917"] + train_dates["put_pnl_0917"]
# train_dates["pnl_0918"] = train_dates["call_pnl_0918"] + train_dates["put_pnl_0918"]

train_dates["pnl_put_at_10"] = train_dates["put_at_0320"] - train_dates["put_at_10"]
train_dates["pnl_put_at_11"] = train_dates["put_at_0320"] - train_dates["put_at_11"]
# train_dates["pnl_put_at_12"] = train_dates["put_at_0320"] - train_dates["put_at_12"]
# train_dates["pnl_put_at_13"] = train_dates["put_at_0320"] - train_dates["put_at_13"]
# train_dates["pnl_put_at_14"] = train_dates["put_at_0320"] - train_dates["put_at_14"]
# train_dates["pnl_put_at_15"] = train_dates["put_at_0320"] - train_dates["put_at_15"]
# train_dates["pnl_put_at_1520"] = train_dates["put_at_0320"] - train_dates["put_at_1520"]


# print(f"PnL 9:15: {train_dates.pnl_0915.sum()}")
# print(f"PnL 9:16: {train_dates.pnl_0916.sum()}")
# print(f"PnL 9:17: {train_dates.pnl_0917.sum()}")
# print(f"PnL 9:18: {train_dates.pnl_0918.sum()}")
# print(f"AVG PnL 9:15: {train_dates.pnl_0915.sum()/train_dates.shape[0]}")

# print(f"Call PnL 9:15: {train_dates.call_pnl_0915.sum()}")
# print(f"Call PnL 9:16: {train_dates.call_pnl_0916.sum()}")
# print(f"Call PnL 9:17: {train_dates.call_pnl_0917.sum()}")
# print(f"Call PnL 9:18: {train_dates.call_pnl_0918.sum()}")
# print(f"AVG Call PnL 9:15: {train_dates.call_pnl_0915.sum()/train_dates.shape[0]}")

print(f"Put PnL 9:15: {train_dates.put_pnl_0915.sum()}")
print(f"Put PnL 9:16: {train_dates.put_pnl_0916.sum()}")
print(f"Put PnL 9:17: {train_dates.put_pnl_0917.sum()}")
print(f"Put PnL 9:18: {train_dates.put_pnl_0918.sum()}")
print(f"AVG Put PnL 9:15: {train_dates.put_pnl_0915.sum()/train_dates.shape[0]}")


print(f"Put pnl_put_at_10: {train_dates.pnl_put_at_10.sum()}")
print(f"Put pnl_put_at_11: {train_dates.pnl_put_at_11.sum()}")
# print(f"Put pnl_put_at_12: {train_dates.pnl_put_at_12.sum()}")
# print(f"Put pnl_put_at_13: {train_dates.pnl_put_at_13.sum()}")
# print(f"Put pnl_put_at_14: {train_dates.pnl_put_at_14.sum()}")
# print(f"Put pnl_put_at_15: {train_dates.pnl_put_at_15.sum()}")
# print(f"Put pnl_put_at_1520: {train_dates.pnl_put_at_1520.sum()}")
print(f"Shape: {train_dates.shape[0]}")

Put PnL 9:15: 1097.6
Put PnL 9:16: 1070.95
Put PnL 9:17: 1172.9499999999998
Put PnL 9:18: 1307.8000000000002
AVG Put PnL 9:15: 13.719999999999999
Put pnl_put_at_10: 1391.3499999999997
Put pnl_put_at_11: 1169.5999999999997
Shape: 80


In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].close

for i in range(-500, 750, 50):
    train_dates["cur_put_atm_strike"] = train_dates["atm_strike"] + i
    key1 = f"put_at_0918_{i}"
    key2 = f"put_at_0320_{i}"
    train_dates[key2] = train_dates.apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.trade_day_before_expiry.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=20)), axis=1)
    train_dates[key1] = train_dates.loc[train_dates[key2].notna()].apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.trade_day_before_expiry.date() + dt.timedelta(days=1), OPTION_TYPE_PUT, dt.time(hour=9, minute=18)), axis=1)
    pnl_key = f"put_pnl_0918_{i}"
    train_dates[pnl_key] = train_dates.loc[train_dates[key2].notna()][key2] - train_dates.loc[train_dates[key2].notna()][key1]
    print(f"Put PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}")

# for i in range(-500, 750, 50):
#     train_dates["cur_put_atm_strike"] = train_dates["atm_strike"] - i
#     key1 = f"call_at_0918_{i}"
#     key2 = f"call_at_0320_{i}"
#     train_dates[key2] = train_dates.apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.trade_day_before_expiry.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=20)), axis=1)
#     train_dates[key1] = train_dates.loc[train_dates[key2].notna()].apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.trade_day_before_expiry.date() + dt.timedelta(days=3), OPTION_TYPE_CALL, dt.time(hour=9, minute=18)), axis=1)
#     pnl_key = f"call_pnl_0918_{i}"
#     train_dates[pnl_key] = train_dates.loc[train_dates[key2].notna()][key2] - train_dates.loc[train_dates[key2].notna()][key1]
#     print(f"Call PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}")

Put PnL at 9:18 for strike: -500, Total: 1.10000000000001, Per Day: 0.01617647058823544, Days: 68
Put PnL at 9:18 for strike: -450, Total: 2.8999999999999946, Per Day: 0.04264705882352933, Days: 68
Put PnL at 9:18 for strike: -400, Total: 6.50000000000001, Per Day: 0.0955882352941178, Days: 68
Put PnL at 9:18 for strike: -350, Total: 10.349999999999984, Per Day: 0.15220588235294094, Days: 68
Put PnL at 9:18 for strike: -300, Total: 19.95000000000002, Per Day: 0.29338235294117676, Days: 68
Put PnL at 9:18 for strike: -250, Total: 34.65000000000001, Per Day: 0.509558823529412, Days: 68
Put PnL at 9:18 for strike: -200, Total: 60.100000000000016, Per Day: 0.8838235294117649, Days: 68
Put PnL at 9:18 for strike: -150, Total: 100.85000000000002, Per Day: 1.483088235294118, Days: 68
Put PnL at 9:18 for strike: -100, Total: 150.25000000000003, Per Day: 2.209558823529412, Days: 68
Put PnL at 9:18 for strike: -50, Total: 214.55, Per Day: 3.1551470588235295, Days: 68
Put PnL at 9:18 for strike: 

2024-02-27 08:04:46,917 - INFO - icharts.py:285 - [139963351557376] - 
file not found: data/options-historical/option-premium/07DEC23-21550-pe.csv
2024-02-27 08:04:46,918 - INFO - icharts.py:43 - [139963351557376] - 
Expiry: 2023-12-07 00:00:00, strike: 21550
2024-02-27 08:04:47,149 - INFO - icharts.py:291 - [139963351557376] - 
data not found on icharts: data/options-historical/option-premium/07DEC23-21550-pe.csv


Put PnL at 9:18 for strike: 600, Total: 251.10000000000048, Per Day: 4.923529411764715, Days: 51


2024-02-27 08:04:49,032 - INFO - icharts.py:285 - [139963351557376] - 
file not found: data/options-historical/option-premium/29DEC22-18750-pe.csv
2024-02-27 08:04:49,032 - INFO - icharts.py:43 - [139963351557376] - 
Expiry: 2022-12-29 00:00:00, strike: 18750
2024-02-27 08:04:49,831 - INFO - icharts.py:285 - [139963351557376] - 
file not found: data/options-historical/option-premium/07DEC23-21600-pe.csv
2024-02-27 08:04:49,831 - INFO - icharts.py:43 - [139963351557376] - 
Expiry: 2023-12-07 00:00:00, strike: 21600
2024-02-27 08:04:49,991 - INFO - icharts.py:291 - [139963351557376] - 
data not found on icharts: data/options-historical/option-premium/07DEC23-21600-pe.csv


Put PnL at 9:18 for strike: 650, Total: 527.3500000000001, Per Day: 10.547000000000002, Days: 50


2024-02-27 08:04:52,075 - INFO - icharts.py:285 - [139963351557376] - 
file not found: data/options-historical/option-premium/29DEC22-18800-pe.csv
2024-02-27 08:04:52,075 - INFO - icharts.py:43 - [139963351557376] - 
Expiry: 2022-12-29 00:00:00, strike: 18800
2024-02-27 08:04:53,043 - INFO - icharts.py:285 - [139963351557376] - 
file not found: data/options-historical/option-premium/07DEC23-21650-pe.csv
2024-02-27 08:04:53,044 - INFO - icharts.py:43 - [139963351557376] - 
Expiry: 2023-12-07 00:00:00, strike: 21650
2024-02-27 08:04:53,203 - INFO - icharts.py:291 - [139963351557376] - 
data not found on icharts: data/options-historical/option-premium/07DEC23-21650-pe.csv


Put PnL at 9:18 for strike: 700, Total: -226.29999999999984, Per Day: -5.657499999999996, Days: 40


In [3]:
train_dates[["trade_day_before_expiry", "atm_strike", "nifty_at_0320", "put_at_0320_250", "put_at_0918_250", "put_pnl_0918_250"]]
# train_dates

,trade_day_before_expiry,atm_strike,nifty_at_0320,put_at_0320_250,put_at_0918_250,put_pnl_0918_250
trade_date,,,,,,
2021-02-03,2021-02-03,14800,14787.50,271.65,281.80,-10.15
2021-03-03,2021-03-03,15250,15259.00,244.65,457.80,-213.15
2021-03-09,2021-03-09,15100,15120.05,241.50,153.05,88.45
2021-03-17,2021-03-17,14700,14714.35,227.35,118.30,109.05
2021-03-24,2021-03-24,14550,14544.80,254.35,320.00,-65.65
2021-03-31,2021-03-31,14700,14686.90,256.85,151.80,105.05
2021-04-28,2021-04-28,14850,14859.20,255.80,102.10,153.70
2021-05-05,2021-05-05,14600,14620.85,220.25,194.95,25.30
2021-05-11,2021-05-11,14850,14850.40,254.20,296.00,-41.80


In [63]:
train_dates[["expiry", "cur_put_atm_strike", "put_at_0320", key, pnl_key]]

,expiry,cur_put_atm_strike,put_at_0320,put_at_0918_50,put_pnl_0918_50
trade_date,,,,,
2021-01-06,2021-01-07,14200,296.15,26.35,269.80
2021-01-13,2021-01-14,14600,281.70,70.75,210.95
2021-01-20,2021-01-21,14700,314.80,27.70,287.10
2021-01-27,2021-01-28,14000,274.25,173.90,100.35
2021-02-10,2021-02-11,15150,271.60,97.95,173.65
2021-02-17,2021-02-18,15250,297.05,66.95,230.10
2021-04-07,2021-04-08,14850,283.40,23.45,259.95
2021-05-19,2021-05-20,15050,266.75,39.55,227.20
2021-05-26,2021-05-27,15350,295.70,67.85,227.85


In [14]:
train_dates.loc[train_dates.pnl_0915 < 0, ["expiry", "atm_strike", "call_at_0320", "call_at_0915", "put_at_0320", "put_at_0915", "call_pnl_0915", "put_pnl_0915", "pnl_0918"]]

,expiry,atm_strike,call_at_0320,call_at_0915,put_at_0320,put_at_0915,call_pnl_0915,put_pnl_0915,pnl_0918
trade_date,,,,,,,,,
2021-01-27,2021-01-28,13950,155.40,63.20,228.05,333.75,92.20,-105.70,-36.45
2021-06-16,2021-06-17,15750,130.40,68.40,247.15,314.35,62.00,-67.20,-6.20
2021-07-07,2021-07-08,15900,90.15,80.15,271.45,292.05,10.00,-20.60,-2.25
2021-10-27,2021-10-28,18200,144.30,99.20,236.50,294.30,45.10,-57.80,-14.40
2021-12-01,2021-12-02,17200,126.40,101.60,263.15,291.60,24.80,-28.45,2.40
2021-12-29,2021-12-30,17200,133.95,93.20,240.05,281.70,40.75,-41.65,4.20
2022-03-09,2022-03-10,16350,183.45,390.70,267.60,64.80,-207.25,202.80,-10.45
2022-05-18,2022-05-19,16250,129.10,13.25,271.05,527.80,115.85,-256.75,-181.80
2022-06-08,2022-06-09,16350,129.40,63.20,257.50,333.95,66.20,-76.45,-7.65
